# Test 02: Import Order Independence

**Purpose:** Verify that after running the auto_update cell, import order no longer matters for cache path configuration.

**The Problem (before fix):**
- If `import transformers` ran before `from introdl import ...`, HuggingFace would lock cache to `~/.cache`
- Environment variables had to be set BEFORE importing any HF/torch libraries

**Expected Results (after fix):**
- Import order should NOT matter
- All cache paths should be correct regardless of import sequence

**Run this on:** Both CoCalc base and Compute Server

---

## IMPORTANT: Restart kernel before running this test!

This test checks that import order doesn't matter. For accurate results, start with a fresh kernel.

In [ ]:
# DS776 Environment Setup & Package Update
# Configures storage paths for proper cleanup/sync, then updates introdl if needed
# If this cell fails, see Lessons/Course_Tools/AUTO_UPDATE_SYSTEM.md for help
%run ../../Lessons/Course_Tools/auto_update_introdl.py

In [ ]:
# Helper function to check cache paths
import os
from pathlib import Path

def check_cache_paths(test_name):
    """Check if all cache paths are configured correctly."""
    home = Path.home()
    bad_cache = str(home / '.cache')
    
    results = []
    
    # Check torch
    import torch
    torch_dir = torch.hub.get_dir()
    results.append(('torch.hub.get_dir()', torch_dir, bad_cache not in torch_dir))
    
    # Check transformers
    from transformers import TRANSFORMERS_CACHE
    results.append(('TRANSFORMERS_CACHE', TRANSFORMERS_CACHE, bad_cache not in TRANSFORMERS_CACHE))
    
    # Check huggingface_hub
    from huggingface_hub import constants as hf_constants
    results.append(('HF_HUB_CACHE', str(hf_constants.HF_HUB_CACHE), bad_cache not in str(hf_constants.HF_HUB_CACHE)))
    
    # Check datasets
    import datasets
    results.append(('HF_DATASETS_CACHE', str(datasets.config.HF_DATASETS_CACHE), bad_cache not in str(datasets.config.HF_DATASETS_CACHE)))
    
    print(f"\n{'='*60}")
    print(f"TEST: {test_name}")
    print(f"{'='*60}")
    
    all_passed = True
    for name, value, passed in results:
        status = "PASS" if passed else "FAIL"
        if not passed:
            all_passed = False
        print(f"{status}: {name}")
        print(f"       {value}")
    
    print(f"\nOVERALL: {'PASS' if all_passed else 'FAIL'}")
    return all_passed

print("Helper function defined. Ready to test import orders.")

## Test A: Import transformers FIRST (before introdl)

This was the problematic case before the fix.

In [ ]:
# Test A: Import transformers first, then introdl
# Note: We already have transformers imported from the helper, 
# but let's check the state

import transformers
print(f"transformers imported from: {transformers.__file__}")
print(f"transformers version: {transformers.__version__}")

# Now import introdl
from introdl import config_paths_keys
print("\nintrodl imported successfully")

# Check paths
check_cache_paths("Import transformers FIRST")

## Test B: Import torch and torchvision FIRST

In [ ]:
# Test B: Import torch/torchvision first
import torch
import torchvision
print(f"torch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")
print(f"torch.hub.get_dir(): {torch.hub.get_dir()}")

# Verify it's not in ~/.cache
from pathlib import Path
home = Path.home()
torch_dir = torch.hub.get_dir()

if str(home / '.cache') in torch_dir:
    print("\nFAIL: torch.hub is using ~/.cache!")
else:
    print("\nPASS: torch.hub is NOT using ~/.cache")

## Test C: Import datasets FIRST

In [ ]:
# Test C: Import datasets first
from datasets import load_dataset
import datasets
print(f"datasets version: {datasets.__version__}")
print(f"HF_DATASETS_CACHE: {datasets.config.HF_DATASETS_CACHE}")

# Verify it's not in ~/.cache
from pathlib import Path
home = Path.home()
ds_cache = str(datasets.config.HF_DATASETS_CACHE)

if str(home / '.cache') in ds_cache:
    print("\nFAIL: datasets cache is using ~/.cache!")
else:
    print("\nPASS: datasets cache is NOT using ~/.cache")

## Test D: Mixed import order (common student pattern)

In [ ]:
# Test D: Common student import pattern
# Students often do random import orders

import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset
import torchvision
from torchvision import transforms
from introdl import config_paths_keys, get_device

print("All imports completed successfully")
print(f"\nDevice: {get_device()}")

# Final check
check_cache_paths("Mixed import order (common student pattern)")

## Summary

If all tests passed:
- Import order no longer matters after running the auto_update cell
- The environment variables are set early enough to affect all library imports

**Next:** Run Test_03 to actually download HuggingFace models and verify they go to the right place.